In [2]:
!pip install numpy pandas scikit-learn matplotlib seaborn streamlit pyngrok

In [3]:
%%writefile data.py
#data generation
import numpy as np
from sklearn.datasets import make_blobs
#data generation
import numpy as np
from sklearn.datasets import make_blobs

def generate_data(n_samples=300, n_features=2, n_clusters=4):
    data,_=make_blobs(n_samples=n_samples,n_features=n_features,centers=n_clusters,random_state=42)
    return data

Writing data.py


In [4]:
%%writefile algorithms.py
#k-means implementation
from sklearn.cluster import KMeans
def apply_kmeans(data,n_clusters):
    kmeans = KMeans(n_clusters=n_clusters,random_state=42)
    labels=kmeans.fit_predict(data)
    centers=kmeans.cluster_centers_
    return labels,centers

Writing algorithms.py


In [5]:
%%writefile visualizer.py
#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
def plot_clusters(data,labels,centers):
  plt.clf() #clear any previous plot
  plt.figure(figsize=(8,6))
  sns.scatterplot(x=data[:,0],y=data[:,1],hue=labels,palette='deep',s=60)
  plt.scatter(centers[:,0],centers[:,1],c='red',marker='X',s=200,label='Centers')
  plt.legend()
  plt.title('K-Means Clustering Visualization')
  plt.grid(True)
  st.pyplot(plt)
def plot_classification(X,y,model):
  import numpy as np
  from matplotlib.colors import ListedColormap
  h=.02
  x_min,x_max=X[:,0].min()-1,X[:,0].max()+1
  y_min,y_max=X[:,1].min()-1,X[:,1].max()+1
  xx,yy=np.meshgrid(np.arange(x_min,x_max,h),np.arange(y_min,y_max,h))
  Z=model.predict(np.c_[xx.ravel(),yy.ravel()])
  Z=Z.reshape(xx.shape)

  plt.clf() #clear any previous plot
  plt.figure(figsize=(8,6))
  cmap_background=ListedColormap(['#FFAAAA','#AAFFAA','#AAAAFF'])
  cmap_points=ListedColormap(['red','blue','green'])
  plt.contourf(xx,yy,Z,alpha=0.8,cmap=cmap_background)
  plt.scatter(X[:,0],X[:,1],c=y,cmap=cmap_points,edgecolor='k')
  plt.title('Classification Visualization')
  plt.grid(True)
  st.pyplot(plt)


Writing visualizer.py


In [15]:
%%writefile app.py
#streamlit app
import streamlit as st
from algorithms import apply_kmeans
from data import generate_data
from visualizer import plot_clusters, plot_classification
#classifiers
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
st.title('Algorithm Visualizer')
#task selection
task=st.sidebar.selectbox('Select Task',['Classification','Clustering','Regression'])
if task=="Clustering":
  st.subheader("K-Means Clustering")
  #sidebar controls
  n_samples=st.sidebar.slider('Number of Samples',100,1000,300)
  n_features=st.sidebar.slider('Number of Features',2,5,2)
  n_clusters=st.sidebar.slider('Number of Clusters',2,10,4)
  #generate data
  data=generate_data(n_samples,n_features,n_clusters)
  #apply k-means
  labels,centers=apply_kmeans(data,n_clusters)
  #display sample data
  st.write("### sample data")
  st.write(data[:5])
  #visualization
  st.write("### Clustering Visualization")
  plot_clusters(data,labels,centers)
elif task=="Classification":
  st.subheader("Classification Algorithms")
  algo_choice= st.selectbox("Select Classification Algorithm",["Logistic Regression","K-Nearest Neighbours","Decision Tree Classifier","SVM","Naive Bayes"])
  n_samples=st.sidebar.slider('Number of Samples',100,1000,300)
  n_features=2 #fixed to 2 for plotting
  n_classes=st.sidebar.slider('Number of Classes',2,3,2)


  #generate data
  X,y=make_classification(n_samples=n_samples,n_features=2,n_informative=2, n_redundant=0, n_classes=n_classes,random_state=42)
  #split data
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
  if algo_choice=="Logistic Regression":
    from sklearn.linear_model import LogisticRegression
    model= LogisticRegression()
  elif algo_choice=="K-Nearest Neighbours":
    k_neighbours=st.sidebar.slider('Number of Neighbours',1,15,3)
    from sklearn.neighbors import KNeighborsClassifier
    model=KNeighborsClassifier(n_neighbors=k_neighbours)
  elif algo_choice=="Decision Tree Classifier":
    from sklearn.tree import DecisionTreeClassifier
    model=DecisionTreeClassifier()
  elif algo_choice=="SVM":
    from sklearn.svm import SVC
    model=SVC(probability=True)#probability true allows better decision boundaries
  elif algo_choice=="Naive Bayes":
    from sklearn.naive_bayes import GaussianNB
    model=GaussianNB()
  #train

  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  st.write(f"###Accuracy: {accuracy_score(y_test,y_pred):.2f}")
  #visualization
  st.write("### Classification Visualization")
  plot_classification(X,y,model)
elif task== "Regression":
  st.subheader("Regression Algorithms")
  algo_choice= st.selectbox("Select Regression Algorithm",["Linear Regression","Decision Tree Regressor"])
  n_samples=st.sidebar.slider('Number of Samples',100,1000,300)
  noise_level=st.sidebar.slider('Noise Level',0.0,10.0,1.0)
  from sklearn.datasets import make_regression
  from sklearn.metrics import mean_squared_error
  from sklearn.model_selection import train_test_split
  import numpy as np
  import matplotlib.pyplot as plt

  X,y=make_regression(n_samples=n_samples,n_features=1,noise=noise_level,random_state=42)
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
  if algo_choice=="Linear Regression":
    from sklearn.linear_model import LinearRegression
    model=LinearRegression()
  elif algo_choice=="Decision Tree Regressor":
    from sklearn.tree import DecisionTreeRegressor
    model=DecisionTreeRegressor()
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  st.write(f"###Mean Squared Error: {mean_squared_error(y_test,y_pred):.2f}")
  #plotting the regression line
  st.write("### Regression Line Visualization")
  fig,ax=plt.subplots()
  ax.scatter(X_test,y_test,color='blue',label='Actual')
  ax.scatter(X_test,y_pred,color='red',label='Predicted')
  ax.set_title("Regression Results")
  ax.legend()
  st.pyplot(fig)

Overwriting app.py


In [16]:
!ngrok config edit
!pkill -f ngrok
!ngrok authtoken 2uWupreoH9ag9c2k4a8EtPuro8a_6qkqhr1DUbKjNUNKVCpzM
!which ngrok

Opening "/root/.config/ngrok/ngrok.yml" with `/bin/bash -c "nano \"/root/.config/ngrok/ngrok.yml\""`
ERROR:  exit status 127


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
/usr/local/bin/ngrok


In [17]:
from pyngrok import ngrok
import threading
#start streamlit in a separate thread
def start_streamlit():
  import os
  os.system('streamlit run app.py')
threading.Thread(target=start_streamlit).start()
#create ngrok tunnel
public_url=ngrok.connect(8501)
print(f"streamlit app is live at: {public_url}")

streamlit app is live at: NgrokTunnel: "https://e27e-34-106-65-108.ngrok-free.app" -> "http://localhost:8501"
